**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Jun  6 16:13:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   csv_test   dataset	'My Drive'   yolov3   Снимок4.PNG


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15069, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15069 (delta 1), reused 7 (delta 0), pack-reused 15056
Receiving objects: 100% (15069/15069), 13.47 MiB | 23.99 MiB/s, done.
Resolving deltas: 100% (10236/10236), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Box" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-06-06 16:26:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  44.1MB/s    in 3.9s    

2021-06-06 16:26:39 (39.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/1_jpg.rf.2aac586533ba280a3f82d6d223aa4149.jpg  
  inflating: data/obj/1_jpg.rf.2aac586533ba280a3f82d6d223aa4149.txt  
  inflating: data/obj/1_jpg.rf.34a22137a2d7565bd66bd6b4767aacc5.jpg  
  inflating: data/obj/1_jpg.rf.34a22137a2d7565bd66bd6b4767aacc5.txt  
  inflating: data/obj/1_jpg.rf.371fa365dbe538aa610c983bc0beb9ce.jpg  
  inflating: data/obj/1_jpg.rf.371fa365dbe538aa610c983bc0beb9ce.txt  
  inflating: data/obj/1_jpg.rf.6087e08ee049d129519aacf4517918b5.jpg  
  inflating: data/obj/1_jpg.rf.6087e08ee049d129519aacf4517918b5.txt  
  inflating: data/obj/1_jpg.rf.8975d4477eea9eace54e807b36857765.jpg  
  inflating: data/obj/1_jpg.rf.8975d4477eea9eace54e807b36857765.txt  
  inflating: data/obj/10_jpg.rf.478e5215e83dd02d3dafb064d8d76f81.jpg  
  inflating: data/obj/10_jpg.rf.478e5215e83dd02d3dafb064d8d76f81.txt  
  inflating: data/obj/10_jpg.rf.54d74264a9b638606b4040c92da1b56e.jpg  
  inflating: data/obj/10_jpg.rf.54d74264a9b638606b

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.5564903846153846 0.5168269230769231 0.3317307692307692 0.4735576923076923
['0', '0.5564903846153846', '0.5168269230769231', '0.3317307692307692', '0.4735576923076923']
0 data/obj/PXL_20201105_173148569_jpg.rf.22a045bf175813f2b27901439111e525.txt
0 0.5564903846153846 0.5168269230769231 0.3317307692307692 0.4735576923076923
0 0.6959134615384616 0.6141826923076923 0.34375 0.21153846153846154
['0', '0.6959134615384616', '0.6141826923076923', '0.34375', '0.21153846153846154']
1 data/obj/16_jpg.rf.716006d8932990f728226303e7d4aa40.txt
0 0.6959134615384616 0.6141826923076923 0.34375 0.21153846153846154
0 0.42908653846153844 0.6298076923076923 0.21634615384615385 0.30288461538461536
['0', '0.42908653846153844', '0.6298076923076923', '0.21634615384615385', '0.30288461538461536']
2 data/obj/PXL_20201107_191150218_jpg.rf.4bcdb342b5a5a12e85aa4c6636609426.txt
0 0.42908653846153844 0.6298076923076923 0.21634615384615385 0.30288461538461536
0 0.59375 0.7487980769230769 0.10336538461538461 0.086538

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/25_jpg.rf.136545cbe252197d0b118656ac657034.jpg', 'data/obj/8_jpg.rf.39d097ce9d72ac95cdd9fc506f4af99f.jpg', 'data/obj/3_jpg.rf.b540bb62b3e31a4b9ef65fd6f3b09383.jpg', 'data/obj/13_jpg.rf.1df0097dfbadf62952f1879702b4dc1f.jpg', 'data/obj/package_2_jpg.rf.8c973fc25e3e5d21b9c530a10fee8b9c.jpg', 'data/obj/23_jpg.rf.4dad4e0808e25edaa22fdd766bc377f6.jpg', 'data/obj/IMG_6823_jpg.rf.0152dabc07ab1fa867917ee49b39bcee.jpg', 'data/obj/IMG_6824_jpg.rf.503fab1e5ae74ba26fca797c54345298.jpg', 'data/obj/PXL_20201107_214721193_jpg.rf.fb52dba4cf2addec4a3d4cc2e9fa527b.jpg', 'data/obj/3_jpg.rf.eb1d9f84db2a88f3d13ae8b6a1d64fd9.jpg', 'data/obj/7_jpg.rf.ee1e513903f5cb2ff0be0a11cfc236f9.jpg', 'data/obj/PXL_20201107_214721193_jpg.rf.2914504d63e6867c2dec7374309c4c03.jpg', 'data/obj/19_jpg.rf.d6839e4fe618d41b12ff637ad0dc8aec.jpg', 'data/obj/package_1_jpg.rf.b38aca10bd8c1c7187ac5202ad8daa7b.jpg', 'data/obj/IMG_3238_jpeg.rf.62fcda3178ff26467074c42b6146a47b.jpg', 'data/obj/IMG_3242_jpeg.rf.98a16de977976bb734

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.398641), count: 4, class_loss = 13.223899, iou_loss = 2.028319, total_loss = 15.252218 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.595526), count: 2, class_loss = 14.424289, iou_loss = 0.385401, total_loss = 14.809690 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 11.127590, iou_loss = 0.000000, total_loss = 11.127590 
 total_bbox = 14292, rewritten_bbox = 0.139938 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.767422), count: 1, class_loss = 12.525836, iou_loss = 0.048948, total_loss = 12.574784 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.240416), count: 4, class_loss = 14.387649, iou_loss = 4.552482, total_loss = 18.940130 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.340424), count: 

In [ ]:
import cv2
import numpy as np
import glob
import random


# Load Yolo
net = cv2.dnn.readNet("yolov3_training_last.weights", "yolov3_testing.cfg")

# Name custom object
classes = ["Box"]

# Images path
images_path = glob.glob(r"/content/gdrive/MyDrive/yolov3/dataset/*.jpg")



layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Insert here the path of your images
random.shuffle(images_path)
# loop through all the images
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 2)


    cv2.imshow("Image", img)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()